# Video Processing for BT Lab
This reads all the log files and refactores them to json format
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import json
import traceback

In [2]:
repo_dir = os.getcwd()

In [3]:
"""
    Extract information from filepath and default parameters
"""

def extract_info_from_filepath(filepaths:list[str]) -> list[dict]:
    all_videos = []
    file_info_from_path = {
        0 : "ID",
        1 : "distance",
        2 : "blanket",
        3 : "breathing",
        4 : "filename"
    }

    video_info = {
        "frames" : None,
        "length" : None,
        "old fps" : None,
        "alias": None,
        "local path" : None,
    }

    splice_stop = 5

    for filepath in filepaths:
        try:
            # keep slashes consistent for splitting
            filepath_no_backslash = filepath.replace("\\", "/")
            filepath_to_list = filepath_no_backslash.split("/")

            # extract info from filepath
            filepath_to_list = filepath_to_list[-splice_stop:]
            patient_info = {tag:filepath_to_list[key] for key, tag in file_info_from_path.items() if key<splice_stop}
            
            # combine information from filepath and default parameters
            patient_full_info = {**patient_info, **video_info}
            patient_full_info["local path"] = filepath

            all_videos.append(patient_full_info) # save to list
        except Exception as e:
            traceback.print_exc()
            print(f"{type(e)} from {filepath}")

    return all_videos

In [4]:
"""
    reads log files

    parameter: filepath to read log files

    return: list of video file paths
"""

def read_log_files(log_directory:str) -> list[str]:
    try:
        os.chdir(log_directory)
    except Exception as e:
        print(type(e))
        return video_files

    log_files = [dir_file for dir_file in os.listdir() if dir_file.endswith(".log")]
    video_files = []
    counter = 0

    for log_file in log_files:
        print(f"Reading - {log_file}")
        try:
            with open(log_file, "r") as log_file_data:
                log_read = log_file_data.read().split("\n")
                if "" in log_read:
                    log_read.remove("")
                video_files.append(log_read)
        except FileNotFoundError as e:
            print(type(e))
        counter += 1

    return video_files

In [5]:
"""
    Switch dictionary values to keep order consistent
"""

def switch_dictionary_values(key1:str, key2:str, patient_information:dict) -> None:
    new_list = []
    for patient in patient_information:
        saved_val = patient[key1]
        patient[key1] = patient[key2]
        patient[key2] = saved_val
        new_list.append(patient)

In [6]:
def export_to_json(patients:list[dict], filename:str) -> None:
    json_to_export = {}
    
    for patient_id, patient_info in enumerate(patients):
        json_to_export[patient_id] = patient_info
    
    export_to_json = json.dumps(json_to_export, indent=2)
    
    with open(filename, "w") as json_data:
        json_data.write(export_to_json)

## Extract information from the exported log files and export to JSON files (stage 1.2)

In [7]:
records_folder = repo_dir + "/records" 

In [8]:
"""
    Read log files
"""

logs_folder = records_folder + "/logs"
video_files = read_log_files(logs_folder)

Reading - DI_CAMERA_P3225-mp4.log
Reading - DI_CAMERA_P3225-png.log
Reading - DI_THERMAL_FLIR-avi.log
Reading - DI_THERMAL_FLIR-png.log
Reading - rgb-exported.log
Reading - rgb_folders-png.log
Reading - thermal_folders-png.log


In [9]:
"""
    make dictionary out of filenames (rgb)
"""
metadata_rgb = extract_info_from_filepath(video_files[0])

In [10]:
"""
    make dictionary out of filenames(thermal)
"""
metadata_thermal = extract_info_from_filepath(video_files[2])
metadata_thermal_flipped = switch_dictionary_values("blanket", "breathing", metadata_thermal)

In [11]:
"""
    export to JSON (rgb)
"""
export_dir = records_folder + "/JSON/rgb.json"
export_to_json(metadata_rgb, export_dir)

In [12]:
"""
    export to JSON (thermal)
"""
export_dir = records_folder + "/JSON/thermal.json"
export_to_json(metadata_thermal, export_dir)